In [1]:
from gnncloudmanufacturing.data import read_fatahi_dataset
from gnncloudmanufacturing.random_solution import random_solve
from gnncloudmanufacturing.validation import total_cost_from_graph, check_feasibility, total_cost_from_gamma
from gnncloudmanufacturing.utils import delta_from_gamma, graph_from_problem, gamma_from_target, delta_from_gamma
from gnncloudmanufacturing.graph_model import GNN, os_type, ss_type

import numpy as np
from tqdm.auto import trange, tqdm
from time import time
import pandas as pd
import torch

In [2]:
dataset = read_fatahi_dataset('../data/fatahi.xlsx')
for problem in dataset:
    print(f'Problem: {problem["name"]}')

  0%|          | 0/18 [00:00<?, ?it/s]

Problem: 5,10,10-1
Problem: 5,10,10-2
Problem: 5,10,10-3
Problem: 10,10,10-1
Problem: 10,10,10-2
Problem: 10,10,10-3
Problem: 5,10,20-1
Problem: 5,10,20-2
Problem: 5,10,20-3
Problem: 5,20,10-1
Problem: 5,20,10-2
Problem: 5,20,10-3
Problem: 5,20,20-1
Problem: 5,20,20-2
Problem: 5,20,20-3
Problem: 5,5,5-1
Problem: 5,5,5-2
Problem: 5,5,5-3


In [8]:
max_operations = 20
model = GNN.load_from_checkpoint(
    checkpoint_path="gnn_ver5.ckpt",
    ins_dim=1,
    ino_dim=max_operations,
    out_dim=16,
    n_layers=2,
    lr=0.001,
)

In [9]:
problem_name = []
total_cost = []
comp_time = []
for problem in tqdm(dataset):
    start = time()
    total = np.inf
    for i in range(5):
        graph = graph_from_problem(problem, max_operations=max_operations)
        graph.edata['feat'][os_type][:, 0] /= 10
        graph.edata['feat'][ss_type][:] /= 100
        pred = model.predict(graph)
        gamma = gamma_from_target(pred, graph, problem)
        delta = delta_from_gamma(problem, gamma)
        check_feasibility(gamma, delta, problem)
        _total = total_cost_from_gamma(problem, gamma, delta).item()
        if total > _total:
            total = _total
    total_cost.append(total)
    problem_name.append(problem['name'])
    comp_time.append(time() - start)

  0%|          | 0/18 [00:00<?, ?it/s]

In [10]:
pd.DataFrame({'problem_name': problem_name, 'total_cost': total_cost, 'comp_time': comp_time}).round(2)

,problem_name,total_cost,comp_time
0,"5,10,10-1",8295.03,0.53
1,"5,10,10-2",16305.10,0.60
2,"5,10,10-3",14223.86,0.59
3,"10,10,10-1",22307.27,0.71
4,"10,10,10-2",25265.64,0.75
5,"10,10,10-3",25035.42,0.80
6,"5,10,20-1",8420.59,1.04
7,"5,10,20-2",13279.76,1.13
8,"5,10,20-3",14028.36,1.19
9,"5,20,10-1",18525.58,0.73
